In [12]:
import pandas as pd
df = pd.read_csv("tmdb_5000_movies.csv")
df.head()
cdf = df.copy()

In [7]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [14]:
df = df.loc[:,["genres", "original_title", "popularity", "revenue", "title", "vote_average", "vote_count"]]

In [16]:
df.head()

,genres,original_title,popularity,revenue,title,vote_average,vote_count
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",Avatar,150.437577,2787965087,Avatar,7.2,11800
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",Pirates of the Caribbean: At World's End,139.082615,961000000,Pirates of the Caribbean: At World's End,6.9,4500
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",Spectre,107.376788,880674609,Spectre,6.3,4466
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",The Dark Knight Rises,112.312950,1084939099,The Dark Knight Rises,7.6,9106
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",John Carter,43.926995,284139100,John Carter,6.1,2124


In [18]:
df["genres"].head()

0    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1    [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
Name: genres, dtype: object

In [25]:
from ast import literal_eval

In [30]:
#df['genres'] = df['genres'].apply(lambda x: [genre.get('name') for genre in literal_eval(x)])
df.head()

,genres,original_title,popularity,revenue,title,vote_average,vote_count
0,"[Action, Adventure, Fantasy, Science Fiction]",Avatar,150.437577,2787965087,Avatar,7.2,11800
1,"[Adventure, Fantasy, Action]",Pirates of the Caribbean: At World's End,139.082615,961000000,Pirates of the Caribbean: At World's End,6.9,4500
2,"[Action, Adventure, Crime]",Spectre,107.376788,880674609,Spectre,6.3,4466
3,"[Action, Crime, Drama, Thriller]",The Dark Knight Rises,112.312950,1084939099,The Dark Knight Rises,7.6,9106
4,"[Action, Adventure, Science Fiction]",John Carter,43.926995,284139100,John Carter,6.1,2124


In [32]:
df.isna().sum()

genres            0
original_title    0
popularity        0
revenue           0
title             0
vote_average      0
vote_count        0
dtype: int64

In [34]:
df.shape

(4803, 7)

In [36]:
df.corr(numeric_only=True)

,popularity,revenue,vote_average,vote_count
popularity,1.000000,0.644724,0.273952,0.778130
revenue,0.644724,1.000000,0.197150,0.781487
vote_average,0.273952,0.197150,1.000000,0.312997
vote_count,0.778130,0.781487,0.312997,1.000000


In [38]:
df.sample()


,genres,original_title,popularity,revenue,title,vote_average,vote_count
1133,"[Action, Crime, Thriller]",15 Minutes,15.205371,56359980,15 Minutes,5.7,191


In [40]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Assuming you have a DataFrame named 'df' with the given features
# For demonstration purposes, let's assume 'original_title' is the movie title column

# Create a copy of the DataFrame with relevant columns
movies = df[['original_title', 'genres', 'popularity', 'vote_average']]

# Fill any missing values (you may want to handle missing values more appropriately)
movies = movies.fillna('')

# Convert the 'genres' column to a string representation
movies['genres_str'] = movies['genres'].apply(lambda x: ' '.join(x))

# Combine features into a single string for each movie
movies['combined_features'] = movies['genres_str'] + ' ' + movies['popularity'].astype(str) + ' ' + movies['vote_average'].astype(str)

# Use CountVectorizer to convert the combined features into a matrix of token counts
count_matrix = CountVectorizer().fit_transform(movies['combined_features'])

# Calculate the cosine similarity between movies
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# Function to get movie recommendations based on the entered movie title
def get_recommendations(movie_title):
    # Check if the entered movie title is in the DataFrame
    if movie_title not in movies['original_title'].values:
        return "Movie not found in the database."

    # Get the index of the entered movie title
    idx = movies.index[movies['original_title'] == movie_title].tolist()[0]

    # Calculate the cosine similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Exclude the movie itself and get top 10 similar movies

    # Get the indices of recommended movies
    movie_indices = [i[0] for i in sim_scores]

    # Return the titles of recommended movies
    return movies['original_title'].iloc[movie_indices]

# Example: User enters a movie title, and we recommend similar movies
user_entered_movie = input("Enter the title of the movie you recently watched: ")
recommended_movies = get_recommendations(user_entered_movie)
print("Recommended movies:")
print(recommended_movies)


Enter the title of the movie you recently watched: Avatar
Recommended movies:
3494    Beastmaster 2: Through the Portal of Time
10                               Superman Returns
14                                   Man of Steel
46                     X-Men: Days of Future Past
61                              Jupiter Ascending
232                                 The Wolverine
813                                      Superman
870                                   Superman II
1230                                   The Shadow
1932                                       Sheena
Name: original_title, dtype: object


In [46]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

movies = df[['original_title', 'genres', 'popularity', 'vote_average']]


movies = movies.fillna('')


movies['genres_str'] = movies['genres'].apply(lambda x: ' '.join(x))

# Combine features into a single string for each movie
movies['combined_features'] = movies['genres_str'] + ' ' + movies['popularity'].astype(str) + ' ' + movies['vote_average'].astype(str)


count_matrix = CountVectorizer().fit_transform(movies['combined_features'])


cosine_sim = cosine_similarity(count_matrix, count_matrix)


def get_recommendations(movie_title):
    
    if movie_title not in movies['original_title'].values:
        return "Movie not found in the database."

    
    idx = movies.index[movies['original_title'] == movie_title].tolist()[0]

    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Exclude the movie itself and get top 10 similar movies

    
    movie_indices = [i[0] for i in sim_scores]

    
    return movies['original_title'].iloc[movie_indices].reset_index(drop=True)

user_entered_movie = input("Enter the title of the movie you recently watched: ")
recommended_movies = get_recommendations(user_entered_movie)
print("Recommended movies:")
print(recommended_movies)


Enter the title of the movie you recently watched: Superman
Recommended movies:
0                 Independence Day: Resurgence
1    Beastmaster 2: Through the Portal of Time
2                                       Avatar
3                             Superman Returns
4                                 Man of Steel
5                   X-Men: Days of Future Past
6                            Jupiter Ascending
7                                The Wolverine
8                                  Superman II
9                                   The Shadow
Name: original_title, dtype: object


In [44]:
count_matrix

<4803x4954 sparse matrix of type '<class 'numpy.int64'>'
	with 20273 stored elements in Compressed Sparse Row format>